# Collinear spin and magnetic systems

In this example we consider iron in the BCC phase.
To show that this material is ferromagnetic we will model it once
allowing collinear spin polarization and once without
and compare the resulting SCF energies. In particular
the ground state can only be found if collinear spins are allowed.

First we setup BCC iron without spin polarization
using a single iron atom inside the unit cell.

In [1]:
using DFTK

a = 5.42352  # Bohr
lattice = a / 2 * [[-1  1  1];
                   [ 1 -1  1];
                   [ 1  1 -1]]
Fe = ElementPsp(:Fe, psp=load_psp("hgh/lda/Fe-q8.hgh"))
atoms = [Fe => [zeros(3)]];

To get the ground-state energy we use an LDA model and rather moderate
discretisation parameters.

In [2]:
kgrid = [3, 3, 3]  # k-point grid (Regular Monkhorst-Pack grid)
Ecut = 15          # kinetic energy cutoff in Hartree
model_nospin = model_LDA(lattice, atoms, temperature=0.01)
basis_nospin = PlaneWaveBasis(model_nospin, Ecut; kgrid=kgrid)

scfres_nospin = self_consistent_field(basis_nospin, tol=1e-6, mixing=KerkerMixing())

scfres_nospin.energies

n     Free energy       Eₙ-Eₙ₋₁     ρout-ρin   Diag
---   ---------------   ---------   --------   ----
  1   -16.64728554505         NaN   3.28e-01    5.0 
  2   -16.64778098215   -4.95e-04   7.78e-02    2.0 
  3   -16.64782811034   -4.71e-05   3.29e-03    1.0 
  4   -16.64783300903   -4.90e-06   1.72e-03    2.8 
  5   -16.64783420779   -1.20e-06   3.33e-04    1.0 
  6   -16.64783430559   -9.78e-08   1.02e-04    2.0 


Energy breakdown:
    Kinetic             15.9217429
    AtomicLocal         -5.0697522
    AtomicNonlocal      -5.2207793
    Ewald               -21.4723040
    PspCorrection       1.8758831 
    Hartree             0.7794653 
    Xc                  -3.4438022
    Entropy             -0.0182880

    total               -16.647834305595


Since we did not specify any initial magnetic moment on the iron atom,
DFTK will automatically assume that a calculation with only spin-paired
electrons should be performed. As a result the obtained ground state
features no spin-polarization.

Now we repeat the calculation, but give the iron atom an initial magnetic moment.
For specifying the magnetic moment pass the desired excess of spin-up over spin-down
electrons at each centre to the `Model` and the guess density functions.
In this case we seek the state with as many spin-parallel
``d``-electrons as possible. In our pseudopotential model the 8 valence
electrons are 2 pair of ``s``-electrons, 1 pair of ``d``-electrons
and 4 unpaired ``d``-electrons giving a desired magnetic moment of `4` at the iron centre.
The structure (i.e. pair mapping and order) of the `magnetic_moments` array needs to agree
with the `atoms` array and `0` magnetic moments need to be specified as well.

In [3]:
magnetic_moments = [Fe => [4, ]]

1-element Array{Pair{ElementPsp,Array{Int64,1}},1}:
 ElementPsp(26, :Fe, PspHgh(8, 0.61, [0.0, 0.0, 0.0, 0.0], 2, [0.454482, 0.63890282, 0.30873177], [[3.01664046 -1.00040646 0.79478164; -1.00040646 2.58303836 -2.05211737; 0.79478164 -2.05211737 3.25763534], [1.49964199 -0.13812935; -0.13812935 0.32687369], [-9.14535371]], "hgh/lda/fe-q8.hgh", "Fe GTH-PADE-q8 GTH-LDA-q8")) => [4]

!!! tip "Units of the magnetisation and magnetic moments in DFTK"
    Unlike all other quantities magnetisation and magnetic moments in DFTK
    are given in units of the Bohr magneton ``μ_B``, which in atomic units has the
    value ``\frac{1}{2}``. Since ``μ_B`` is the magnetic moment of a single electron
    the advantage is that one can directly think of these quantities as the excess of
    spin-up electrons or spin-up electron density.

We repeat the calculation using the same model as before. DFTK now detects
the non-zero moment and switches to a collinear calculation.

In [4]:
model = model_LDA(lattice, atoms, magnetic_moments=magnetic_moments, temperature=0.01)
basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid)
ρspin = guess_spin_density(basis, magnetic_moments)
scfres = self_consistent_field(basis, tol=1e-10, ρspin=ρspin, mixing=KerkerMixing())

scfres.energies

n     Free energy       Eₙ-Eₙ₋₁     ρout-ρin   Magnet   Diag
---   ---------------   ---------   --------   ------   ----
  1   -16.65997408942         NaN   3.41e-01    2.620    4.4 
  2   -16.66653772919   -6.56e-03   6.92e-02    2.328    2.5 
  3   -16.66694396071   -4.06e-04   1.05e-02    2.276    1.4 
  4   -16.66698826374   -4.43e-05   1.18e-03    2.275    1.9 
  5   -16.66699400087   -5.74e-06   8.60e-04    2.284    2.8 
  6   -16.66699610033   -2.10e-06   4.29e-04    2.292    1.5 
  7   -16.66699649850   -3.98e-07   7.28e-05    2.296    1.8 
  8   -16.66699652206   -2.36e-08   1.12e-05    2.297    2.6 
  9   -16.66699652212   -6.20e-11   2.07e-06    2.297    2.1 


Energy breakdown:
    Kinetic             16.3015660
    AtomicLocal         -5.2260645
    AtomicNonlocal      -5.4136149
    Ewald               -21.4723040
    PspCorrection       1.8758831 
    Hartree             0.8201416 
    Xc                  -3.5395247
    Entropy             -0.0130791

    total               -16.666996522121


!!! note "Model and magnetic moments"
    DFTK does not store the `magnetic_moments` inside the `Model`, but only uses them
    to determine the lattice symmetries. This step was taken to keep `Model`
    (which contains the physical model) independent of the details of the numerical details
    such as the initial guess for the spin density.

In direct comparison we notice the first, spin-paired calculation to be
a little higher in energy

In [5]:
println("No magnetization: ", scfres_nospin.energies.total)
println("Magnetic case:    ", scfres.energies.total);

No magnetization: -16.647834305594728
Magnetic case:    -16.66699652212134


Notice that with the small cutoffs we use to generate the online
documentation the calculation is far from converged.
With more realistic parameters a larger energy difference of about
0.1 Hartree is obtained.

The spin polarization in the magnetic case is visible if we
consider the occupation of the spin-up and spin-down Kohn-Sham orbitals.
Especially for the ``d``-orbitals these differ rather drastically.
For example for the first ``k``-point:

In [6]:
iup   = 1
idown = iup + length(scfres.basis.kpoints) ÷ 2
@show scfres.occupation[iup]
@show scfres.occupation[idown];

scfres.occupation[iup] = [1.0, 0.999998813918381, 0.9999988139179684, 0.9999988139175023, 0.9585650147127615, 0.9585649831158012, 1.154028975855243e-29, 1.1540288670681591e-29, 1.1540280038943253e-29, 1.3956887522242502e-31, 1.3956884361807453e-31, 1.3956879162922788e-31, 1.0609941884959859e-45, 8.576993324779098e-47, 8.576948207013733e-47]
scfres.occupation[idown] = [1.0, 0.8327424978223893, 0.8327422653489471, 0.832742203156895, 7.846458919378072e-6, 7.846437052329144e-6, 1.5493026231424408e-32, 1.549300388171506e-32, 1.5493000573162046e-32, 9.808690815377012e-34, 9.808682309081347e-34, 9.808656826467027e-34, 6.717090917024839e-49, 6.717081655555281e-49, 3.6115639485301415e-51]


Similarly the eigenvalues differ

In [7]:
@show scfres.eigenvalues[iup]
@show scfres.eigenvalues[idown];

scfres.eigenvalues[iup] = [-0.06926786720399174, 0.35688219276195904, 0.3568821962415984, 0.35688220017112515, 0.46191761720615016, 0.46191762516144685, 1.1596478195121955, 1.1596478204548677, 1.159647827934526, 1.2037982339141615, 1.2037982361785886, 1.20379823990355, 1.5289019633372174, 1.5540548947820498, 1.55405494738543]
scfres.eigenvalues[idown] = [-0.030405543285342944, 0.47727883617180833, 0.4772788528625936, 0.47727885732776365, 0.6108851390574911, 0.6108851669264332, 1.225779915978991, 1.2257799304046593, 1.2257799325401737, 1.25337697882439, 1.2533769874965965, 1.2533770134762814, 1.6025508792637628, 1.6025508930516899, 1.6548077236865042]


!!! note "k-points in collinear calculations"
    For collinear calculations the `kpoints` field of the `PlaneWaveBasis` object contains
    each ``k``-point coordinate twice, once associated with spin-up and once with down-down.
    The list first contains all spin-up ``k``-points and then all spin-down ``k``-points,
    such that `iup` and `idown` index the same ``k``-point, but differing spins.